In [ ]:
# LSTM 网络


In [1]:
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from imp import reload

import sys

sys.path.append("..")
import d2l_pytorch.d2l as d2l

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

reload(d2l)


D:\Temp\ipykernel_34300\1599870281.py:5: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


<module 'd2l_pytorch.d2l' from '..\\d2l_pytorch\\d2l.py'>

In [2]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics("F:\python_code\DL\Datasets\JayChouLyrics\jaychou_lyrics.txt")

In [6]:
# 从零开始实现

num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print("will use", device)

def get_params():
  def _one(shape):
    ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype = torch.float32)
    return torch.nn.Parameter(ts, requires_grad=True)
  def _three():
    return (_one((num_inputs, num_hiddens)), _one((num_hiddens, num_hiddens)), torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
  
  W_xi, W_hi, b_i = _three()
  W_xf, W_hf, b_f = _three()
  W_xo, W_ho, b_o = _three()
  W_xc, W_hc, b_c = _three()

  W_hq = _one((num_hiddens, num_outputs))
  b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32),requires_grad=True)
  return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

will use cuda


In [5]:
def init_lstm_state(batch_size, num_hiddens, device):
  return (
    torch.zeros((batch_size, num_hiddens), device=device),
    torch.zeros((batch_size, num_hiddens), device=device),
  )


In [7]:
# 根据长短记忆的计算表达式定义模型，需要注意的是，只有隐藏状态会传递到输出层，而记忆细胞不参与输出层的计算


def lstm(inputs, state, params):
  [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
  (H, C) = state
  outputs = []
  for X in inputs:
    I_ = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
    F_ = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
    O_ = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
    C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
    C = F_ * C + I_ * C_tilda
    H = O_ * C.tanh()
    Y = torch.matmul(H, W_hq) + b_q
    outputs.append(Y)

  return outputs, (H, C)


In [8]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ["分开", "不分开"]

In [9]:
d2l.train_and_predict_rnn(
  lstm,
  get_params,
  init_lstm_state,
  num_hiddens,
  vocab_size,
  device,
  corpus_indices,
  idx_to_char,
  char_to_idx,
  False,
  num_epochs,
  num_steps,
  lr,
  clipping_theta,
  batch_size,
  pred_period,
  pred_len,
  prefixes,
)

epoch 40, perplexity 214.819970, time 1.91 sec
 - 分开
我想你的我的可爱
我想想你的我的爱爱
我想你的我的可爱
我想想你的我的爱爱
我想你的我的可爱
我想
 - 不分开
我想你的我的可爱
我想想你的我的爱爱
我想你的我的可爱
我想想你的我的爱爱
我想你的我的可爱
我想
epoch 80, perplexity 70.506629, time 1.21 sec
 - 分开
我想你这你 我不要不
我不你 你不了 我不不不
我说你的你 我不想不你
我说你么 我不要多
我不你
 - 不分开
我想要这 我不要
我不不觉
我不要好 我不要
不不 我不想
我不要这 我不要
不不 我不想
我不要
epoch 120, perplexity 16.585424, time 1.13 sec
 - 分开
我想要这样 我不想
不你 
快情我想想你
你你的让我面著
不知不觉
我已了这节奏
我知好觉
我该了
 - 不分开
你知的美 你在红的可爱女人
我想你这辈 我不你的可女人
你亮 爱怪 在单了停
说有你么 我不多难恼
epoch 160, perplexity 4.214057, time 1.08 sec
 - 分开 我想想和汉堡 
我想想你的微笑 想想想和和汉堡
我想要你的微笑每天都能看到 
我知道这里很美但家的
 - 不分开
你知 我想很久久
我不能 你想我的难样
问通在直不我 抛发到我满的怒火
我想揍你已经 却你依依不舍
